In [1]:
import pandas as pd
import numpy as np

In [2]:
url = 'http://files.grouplens.org/datasets/movielens/ml-100k/u.data'
df = pd.read_csv(url, sep='\t', names=['user_id', 'item_id', 'rating', 'timestamp'])

In [6]:
ratings_matrix = df.pivot_table(index='user_id', columns='item_id', values='rating').fillna(0).values

In [7]:
ratings_matrix.shape

(943, 1682)

In [9]:
temp = ratings_matrix[0]

temp

array([5., 3., 4., ..., 0., 0., 0.])